In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore, auth
from datetime import datetime
import json 
from dotenv import load_dotenv
from random import randint
import os
import re
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

load_dotenv()

cred = credentials.Certificate(os.getenv("firebase_secret"))
app = firebase_admin.initialize_app(cred)


db = firestore.client()


def load_json_file(filename):
    with open(filename) as fp:
        content = json.load(fp)
    return content


In [2]:
DIFF = load_json_file("diff.json")

In [4]:
def return_old_new_pair(dictionary: dict, key: str = ''): 
    if 'd1' in dictionary:
        return (dictionary['d1'], dictionary['d2'])
    else:

        return {key: return_old_new_pair({key: val['d1'], key: val['d2']}, key = key ) for (key,val) in dictionary.items()}

In [5]:
def flatten_dict(dicionary: dict) -> dict:
    flat_dict = {}
    for key, val in dicionary.items():
        if type(val) == dict:
            for sub_key, sub_val in flatten_dict(val).items():
                flat_dict[key + '.' + sub_key] = sub_val
        else:
            flat_dict[key] = val
    return flat_dict

In [ ]:
from datetime import datetime


In [24]:
MAPA_VALORES_MODIFICADOS = {'ambiente': lambda x: db.document(f"ambientes/{x}"),
 'detalhes.descricao': lambda x: x,
 'detalhes.n_serie': lambda x: x,
 'detalhes.transf_local': lambda x: datetime.strptime(x, "%d/%m/%Y") if x else None,
 'edificio': lambda x: x,
 'origem': lambda x: x,
 'responsavel': lambda x: x}

## Modificados

In [32]:
TO_MODIFY = {} 

for patrimonio, value in DIFF['modified'].items():
    novos_valores = flatten_dict(value)
    update_dict = {}
    for key, value in novos_valores.items():
        if 'd1' in key:
            new_key = key.replace(".d1", "") 
            update_dict[new_key] = value
    docref = db.document(f'items/{patrimonio}')
    docdata = docref.get().to_dict()
    TO_MODIFY[patrimonio] = {'atual': docdata, 'update': update_dict} 
    update_dict_melhorado = {key: MAPA_VALORES_MODIFICADOS[key](value) for (key, value) in update_dict.items()}
     
    docref.update(update_dict_melhorado)


In [31]:
import json
from pathlib import Path

updated_path = Path("json0929/updated.json")

with open(updated_path, 'w') as fp:
    json.dump(TO_MODIFY, fp, ensure_ascii=False, indent=2, default=str)

## Criados

In [ ]:
MAPA_VALORES_MODIFICADOS = {'ambiente': lambda x: db.document(f"ambientes/{x}"),
 'detalhes.descricao': lambda x: x,
 'detalhes.n_serie': lambda x: x,
 'detalhes.transf_local': lambda x: datetime.strptime(x, "%d/%m/%Y") if x else None,
 'edificio': lambda x: x,
 'origem': lambda x: x,
 'responsavel': lambda x: x}

In [3]:
from datetime import datetime

In [44]:
DIFF = load_json_file("diff.json")

In [45]:
def processar_dicionario(dicionario):
    dicionario["ambiente"] = db.document(f"ambientes/{dicionario['ambiente']}")
    if dicionario["detalhes"]["incorporacao"]: 
        dicionario["detalhes"]["incorporacao"] = datetime.strptime(dicionario["detalhes"]["incorporacao"], "%d/%m/%Y") 
    if dicionario["detalhes"]["transf_local"]:
        dicionario["detalhes"]["transf_local"] = datetime.strptime(dicionario["detalhes"]["transf_local"], "%d/%m/%Y")
    dicionario['meta'] = {"inteiro": True, "partes": [], "updated": datetime.now(), "volumado": False, "volume": None}
    
for patrimonio, dicionario in DIFF['new'].items():
    processar_dicionario(dicionario) 

In [47]:
for patrimonio, dicionario in DIFF['new'].items():
    docref = db.document(f'items/{patrimonio}')  
    print(dicionario)
    docref.set(dicionario)

{'responsavel': 'Rafael dos Reis Abreu', 'ambiente': <google.cloud.firestore_v1.document.DocumentReference object at 0x7f36f034e590>, 'edificio': 'BNU03 - Bloco Acadêmico B', 'detalhes': {'patrimonio': 1021814, 'n_controle': None, 'cod_barras': 364329, 'n_serie': 1675064, 'descricao': 'BEBEDOURO DE ÁGUA  - TIPO VERTICAL FIXO. MARCA: LIBELL; MODELO: PRESS STAR INOX. BEBEDOURO TIPO PRESSÃO; COM DUAS BICAS (TORNEIRAS) UMA DO TIPO COPO E UMA DO TIPO JATO; ALIMENTADO COM ÁGUA DA REDE; PIA EM AÇO INOXIDÁVEL E RESERVATÓRIO DE ÁGUA EM POLIACETAL; 220 V.', 'conservacao': 'EM USO', 'incorporacao': datetime.datetime(2023, 9, 20, 0, 0), 'transf_local': None, 'valor': 695, 'situacao': 'Ativo', 'peso': '', 'medidas': ''}, 'short_descricao': 'BEBEDOURO DE ÁGUA ', 'origem': 'B026', 'key': 364329, 'tipo': 'Permanente'}
{'responsavel': 'Rafael dos Reis Abreu', 'ambiente': <google.cloud.firestore_v1.document.DocumentReference object at 0x7f36f034ff40>, 'edificio': 'BNU03 - Bloco Acadêmico B', 'detalhes':

In [48]:
len(DIFF['new'])

41

In [49]:
len(DIFF['modified'])

257